# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [58]:
!pip install scipy==1.1.0 

     |████████████████████████████████| 31.2MB 102kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2


We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## unzip files 
!unzip /content/drive/MyDrive/freelancer_dataset/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [60]:
#train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 16 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [61]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,1,2,3,5,9,13,17,21,25,27,28,29]
#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            x=len(img_idx)
            #batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,x,100,100,3))
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    #cropping an image if size height or width is 120* 160
                    height, width , channel = image.shape
                    if height == 120 or width == 120:
                        image=image[20:140,:120,:]
                    #crop the images and resize them. Note that the images are of 2 different shape                     
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #resizing all iamge to 100*100
                    image = cv2.resize(image,(100,100))
                    image_r = image
                    image_r[:,:,1:2] = 0
                    batch_data[folder,idx,:,:,0] = (image_r[:,:,0] - image_r.mean())/image_r.std()
                    #normalise and feed in the image
                    image_g = image
                    image_g[:,:,0:2] = 0
                    batch_data[folder,idx,:,:,1] = (image_g[:,:,1] - image_g.mean())/image_g.std()
                    #normalise and feed in the image
                    image_b = image
                    image_b[:,:,0:1] = 0
                    batch_data[folder,idx,:,:,2] = (image_b[:,:,2] - image_b.mean())/image_b.std()
                    #normalise and feed in the image
                
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        if len(folder_list) > num_batches * batch_size:
            x=len(img_idx)
            remaining_data_size = len(folder_list) - (num_batches * batch_size)
            batch_data = np.zeros((remaining_data_size,x,100,100,3))
            batch_labels = np.zeros((remaining_data_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(remaining_data_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + ((num_batches+1)*remaining_data_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + ((num_batches+1)*remaining_data_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                     
                    #cropping an image if size height or width is 120* 160
                    height, width , channel = image.shape
                    if height == 120 or width == 120:
                        image=image[20:140,:120,:]
                    #crop the images and resize them. Note that the images are of 2 different shape                     
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    #resizing all iamge to 100*100
                    image = cv2.resize(image,(100,100))
                    image_r = image
                    image_r[:,:,1:2] = 0
                    batch_data[folder,idx,:,:,0] = (image_r[:,:,0] - image_r.mean())/image_r.std()
                    #normalise and feed in the image
                    image_g = image
                    image_g[:,:,0:2] = 0
                    batch_data[folder,idx,:,:,1] = (image_g[:,:,1] - image_g.mean())/image_g.std()
                    #normalise and feed in the image
                    image_b = image
                    image_b[:,:,0:1] = 0
                    batch_data[folder,idx,:,:,2] = (image_b[:,:,2] - image_b.mean())/image_b.std()
                    #normalise and feed in the image
                
                batch_labels[folder, int(t[folder + ((num_batches+1)*remaining_data_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [62]:
curr_dt_time = datetime.datetime.now()
# train_path = '/notebooks/storage/Final_data/Collated_training/train'
# val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [63]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
#Our import
from keras.regularizers import l2

In [34]:
# ## change size to 150 * 150 
# #write your model here
# model = Sequential()

# model.add(Conv3D(32, (3, 3, 3), padding='same', input_shape=(19,100,100,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu')) #can use elu as well
# model.add(MaxPooling3D(pool_size=(2,2,2)))

# model.add(Conv3D(64, (3, 3, 3), padding='same', input_shape=(19,100,100,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu')) #can use elu as well
# model.add(MaxPooling3D(pool_size=(2,2,2)))

# model.add(Conv3D(128, (3, 3, 3), padding='same', input_shape=(19,100,100,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu')) #can use elu as well
# model.add(MaxPooling3D(pool_size=(2,2,2)))

# model.add(Flatten())
# model.add(Dense(512,kernel_regularizer=l2(0.01)))
# model.add(Dropout(0.5))
# model.add(Activation('relu'))
# model.add(Dense(5)) #Look at dropout
# model.add(Activation('softmax')

In [64]:
#write your model here
num_classes = 5
model = Sequential()

model.add(Conv3D(32, (3, 3, 3), padding='same', input_shape=(13,100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding='same', input_shape=(13,100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(128, (3, 3, 3), padding='same', input_shape=(13,100,100,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [65]:
optimiser = 'SGD' #write your optimizer #can try new ones like ADAMS or NADAMS (for future)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_18 (Conv3D)           (None, 13, 100, 100, 32)  2624      
_________________________________________________________________
activation_30 (Activation)   (None, 13, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 13, 100, 100, 32)  128       
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 6, 50, 50, 32)     0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 6, 50, 50, 32)     0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 6, 50, 50, 64)     55360     
_________________________________________________________________
activation_31 (Activation)   (None, 6, 50, 50, 64)    

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [66]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


In [57]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001) # write the REducelronplateau code here
#try different patience level

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [67]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [68]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=3, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.


Source path =  /content/Project_data/train ; batch size = 16
Epoch 1/3
41/42 [============================>.] - ETA: 0s - loss: 22.5057 - categorical_accuracy: 0.2734

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.


42/42 [==============================] - 29s 687ms/step - loss: 22.1920 - categorical_accuracy: 0.2765 - val_loss: 11.4362 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2021-03-2111_44_27.131740/model-00001-15.76051-0.33937-11.43622-0.29000.h5
Epoch 2/3
42/42 [==============================] - 27s 649ms/step - loss: 11.0903 - categorical_accuracy: 0.5435 - val_loss: 11.4644 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2021-03-2111_44_27.131740/model-00002-11.01499-0.54600-11.46440-0.23000.h5
Epoch 3/3
42/42 [==============================] - 27s 661ms/step - loss: 10.7753 - categorical_accuracy: 0.5788 - val_loss: 11.6309 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model_init_2021-03-2111_44_27.131740/model-00003-10.71179-0.58824-11.63087-0.24000.h5
